In [24]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import spark_partition_id, asc, desc
from pyspark.sql.functions  import spark_partition_id
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer
sc.setSystemProperty("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
# spark.serializer org.apache.spark.serializer.KryoSerializer
# http://spark-configuration.luminousmen.com/
# https://spoddutur.github.io/spark-notes/distribution_of_executors_cores_and_memory_for_spark_application.html
# https://www.codetd.com/pt/article/10830437
# https://databricks.com/session_na20/fine-tuning-and-enhancing-performance-of-apache-spark-jobs

In [ ]:
# FIFO E FEAR
# tunning
# SPARK UI, paralelismo, particionamento, joins, serialização

In [ ]:
# spark.default.parallelism=140
# spark.executor.memory=9g
# spark.executor.instances=14
# spark.driver.cores=5
# spark.executor.cores=5
# spark.driver.memory=9g
# spark.driver.maxResultSize=9g
# spark.driver.memoryOverhead=921m
# spark.executor.memoryOverhead=921m
# spark.dynamicAllocation.enabled=false
# spark.sql.adaptive.enabled=true
# spark.memory.fraction=0.8
# spark.scheduler.barrier.maxConcurrentTasksCheck.maxFailures=5
# spark.rdd.compress=true
# spark.shuffle.compress=true
# spark.shuffle.spill.compress=true
# spark.serializer=org.apache.spark.serializer.KryoSerializer
# spark.executor.extraJavaOptions=-XX:+UseG1GC -XX:+G1SummarizeConcMark
# spark.driver.extraJavaOptions=-XX:+UseG1GC -XX:+G1SummarizeConcMark

In [14]:
# from pyspark import SparkContext
# # Start the spark context using the SparkConf object named `conf` the extension created in your kernel.
# sc=SparkContext.getOrCreate(conf=conf)

# from pyspark.sql import SparkSession
# spark = SparkSession.builder\
#         .config('spark.extraListeners', 'sparkmonitor.listener.JupyterSparkMonitorListener')\
#         .config('spark.driver.extraClassPath', 'venv/lib/python3.7/site-packages/sparkmonitor/listener.jar')\
#         .getOrCreate()



# SKEW - analise de spark UI

    spark.drives.cores
    spark.driver.memory
    executor-memory
    executor-cores
    spark.cores.max
    
    muita memória por executor pode resultar em atrasos excessivos no GC
    pouca memória pode perder os benefícios da execução de várias tarefas em uma única JVM
    analisar network, CPU, memoria
    tempo de execução em cada task
    numero de registros no dataframe
    memoria e particonamento
    
    FIFO - primeiro a entrar, primeiro a sair
    FAIR > FIFO

In [21]:
df = spark.createDataFrame([
    Row(a=1, b=4., c='GFG1', d=date(2000, 8, 1),
        e=datetime(2000, 8, 1, 12, 0)),
   
    Row(a=2, b=8., c='GFG2', d=date(2000, 6, 2),
        e=datetime(2000, 6, 2, 12, 0)),
   
    Row(a=4, b=5., c='GFG3', d=date(2000, 5, 3),
        e=datetime(2000, 5, 3, 12, 0))
])
 
# show table
df.show()
 
# show schema
df.printSchema()

# numero de partioções
n_partitions = df.rdd.getNumPartitions()
print(f'Numero de partições: {n_partitions}\n')

print('Numero de linhas por partição')
df.withColumn("partitionId", spark_partition_id()).groupBy("partitionId").count().show()

# df.withColumn("a", spark_partition_id())\
#     .groupBy("a")\
#     .count()\
#     .orderBy(asc("count"))\
#     .show()

def _to_java_object_rdd(rdd):  
    """ Return a JavaRDD of Object by unpickling
    It will convert each Python object into Java object by Pyrolite, whenever the
    RDD is serialized in batch or not.
    """
    rdd = rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
    return rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)

JavaObj = _to_java_object_rdd(df.rdd)

nbytes = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(JavaObj)
mb = round(int(nbytes / df.rdd.getNumPartitions()) / 1024 / 1024, 2)
print(f'Tamanho aproximado do dataframe: {mb} MB')
size_partitions = mb / n_partitions
print(f'Tamanho aproximado de cada partição: {size_partitions} MB')
print('\nspark ui ->')
print(sc.uiWebUrl)
# dados reais
# working...


+---+---+----+----------+-------------------+
|  a|  b|   c|         d|                  e|
+---+---+----+----------+-------------------+
|  1|4.0|GFG1|2000-08-01|2000-08-01 12:00:00|
|  2|8.0|GFG2|2000-06-02|2000-06-02 12:00:00|
|  4|5.0|GFG3|2000-05-03|2000-05-03 12:00:00|
+---+---+----+----------+-------------------+

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)

Numero de partições: 8

Numero de linhas por partição
+-----------+-----+
|partitionId|count|
+-----------+-----+
|          5|    1|
|          7|    1|
|          2|    1|
+-----------+-----+

Tamanho aproximado do dataframe: 2.8 MB
Tamanho aproximado de cada partição: 0.35 MB

spark ui ->
http://192.168.0.15:4040


In [22]:
spark.sparkContext.getConf().getAll()

[('spark.app.startTime', '1644177423398'),
 ('spark.app.id', 'local-1644177424543'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', '192.168.0.15'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.warehouse.dir', 'file:/home/magico/Main/spark-warehouse'),
 ('spark.driver.port', '33461'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]